### Analiza danych dotyczących rezerwacji hotelowych
##### Wykorzystane dane: **https://www.kaggle.com/jessemostipak/hotel-booking-demand**

In [0]:
import pyspark.sql.functions as f

In [0]:
#Wczytanie danych
data = spark.read.format('com.databricks.spark.csv').options(inferSchema='true', header='true', delimiter=',').load('/FileStore/tables/hotel_bookings.csv')

In [0]:
display(data.head(5))

hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
Resort Hotel,0,342,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01
Resort Hotel,0,737,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01
Resort Hotel,0,7,2015,July,27,1,0,1,1,0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NULL,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02
Resort Hotel,0,13,2015,July,27,1,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02
Resort Hotel,0,14,2015,July,27,1,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,NULL,0,Transient,98.0,0,1,Check-Out,2015-07-03


#### Jaka część rezerwacji zostanie odwołana? - skala problemu

In [0]:
data = data.withColumn('is_canceled', f.when(data.is_canceled ==1, 'Yes').otherwise('No'))
data = data.withColumnRenamed("is_canceled", "canceled")
display(data.groupby('canceled').count())

canceled,count
No,75166
Yes,44224


Biorąc pod uwagę, że aż 37% rezerwacji zostaje odwołanych, zjawisko to wydaje sie być bardzo istotnym elementem na przykład w trakcie planowania obłożenia hoteli. W raporcie dokonam eksploracyjnej analizy danych.

### Przygotowanie i wstępna analiza danych

In [0]:
display(data.summary())

summary,hotel,canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
count,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390
mean,null,null,104.01141636652986,2016.156554150264,null,27.16517296255968,15.798241058715135,0.9275986263506156,2.500301532791691,1.8564033838679956,0.10388990333874994,0.007948739425412514,null,null,null,null,0.03191222045397437,0.08711784906608594,0.13709690928888515,null,null,0.22112404724013737,null,86.69338185346919,189.26673532440782,2.321149174972778,null,101.83112153446673,0.06251779881062065,0.5713627607002262,null,null
stddev,null,null,106.86309704798795,0.7074759445193511,null,13.6051383554977,8.780829470578352,0.9986134945978756,1.9082856150479124,0.5792609988327547,0.39856144478644184,0.09743619130126424,null,null,null,null,0.1757671454106565,0.8443363841545128,1.4974368477076807,null,null,0.6523055726747704,null,110.7745476429513,131.6550146385122,17.59472087877625,null,50.53579028554864,0.2452911474674931,0.7927984228094103,null,null
min,City Hotel,No,0,2015,April,1,1,0,0,0,0,0,BB,ABW,Aviation,Corporate,0,0,0,A,A,0,No Deposit,1,10,0,Contract,-6.38,0,0,Canceled,2014-10-17
25%,null,null,18,2016,null,16,8,0,1,2,0.0,0,null,null,null,null,0,0,0,null,null,0,null,9.0,62.0,0,null,69.25,0,0,null,null
50%,null,null,69,2016,null,28,16,1,2,2,0.0,0,null,null,null,null,0,0,0,null,null,0,null,14.0,178.0,0,null,94.5,0,0,null,null
75%,null,null,160,2017,null,38,23,2,3,2,0.0,0,null,null,null,null,0,0,0,null,null,0,null,229.0,270.0,0,null,126.0,0,1,null,null
max,Resort Hotel,Yes,737,2017,September,53,31,19,50,55,NA,10,Undefined,ZWE,Undefined,Undefined,1,26,72,P,P,21,Refundable,NULL,NULL,391,Transient-Party,5400.0,8,5,No-Show,2017-09-14


Zbiór danych składa się z 119390 pojedynczych rezerwacji oraz 32 zmiennych, zarówno numerycznych, jak i kategorycznych. Zbiór nie zawiera braków danych. Opis wybranych zmiennych:
- hotel - dane pochodzą z dwócho hoteli (Resort Hotel, City Hotel)
- canceled - zmienna binarna informująca czy rezerwacja została odwołana (1=tak, 0=nie)
- lead_time - czas w dniach pomiędzy daną rezerwacji, a datą planowanego przyjazdu
- children - liczba dzieci
- country - kraj z którego pochodziła rezerwacja
- deposit_type - czy rezerwujący musiał umieścic depozyt i czy był on zwracany w razie odwołania rezerwacji
- stays_in_weekend - liczba nocy weekendowych w zarezerwowanym terminie
- stays_in_week - liczbe nocy od poniedziałku do piątku z zarezerwowanym terminie

In [0]:
#dane posiadają stringi "NULL" oraz "NA" w miejscu braków - zamieniam na logiczny null
for column in data.columns:
  data = data.withColumn(column, f.when(f.col(column).isin('NULL','NA'), None).otherwise(f.col(column)))

In [0]:
for col in data.columns:
  null_count = data.filter(data[col].isNull()).count()
  if (null_count > 0):
      print(col,':',null_count)

children : 4
country : 488
agent : 16340
company : 112593

Zdecydowałem się usunąć kolumny agent oraz company ze względu na bardzo dużą ilość braków danych, szczególnie w przypadku kolumny company. 
Zmienna country posiada natomiast jedynie 488 braków, a children jedynie 4. Z uwagi na fakt, że jest to niewielka liczba biorąc pod uwagę skalę danych, usuwam te rekordy.

In [0]:
data = data.drop('company','agent')
data = data.na.drop(subset=['country','children'])
display(data.head(5))

hotel,canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
Resort Hotel,No,342,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,0,Transient,0.0,0,0,Check-Out,2015-07-01
Resort Hotel,No,737,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,0,Transient,0.0,0,0,Check-Out,2015-07-01
Resort Hotel,No,7,2015,July,27,1,0,1,1,0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,0,Transient,75.0,0,0,Check-Out,2015-07-02
Resort Hotel,No,13,2015,July,27,1,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,0,Transient,75.0,0,0,Check-Out,2015-07-02
Resort Hotel,No,14,2015,July,27,1,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [0]:
display(data.groupby('hotel').count())

hotel,count
Resort Hotel,39596
City Hotel,79302


Niecałe 40 tysięcy obserwacji pochodzi z Resort Hotel, a prawie 80 tysięcy z City Hotel.

#### Kraj pochodzenia rezerwujących

In [0]:
countries = data.groupby('country').count().orderBy("count", ascending=False)
countries.createOrReplaceTempView('countries')

In [0]:
sql_query = "select country,count, count/{} as percentage from countries".format(data.count())
d = spark.sql(sql_query)
d.withColumn('percentage', f.round(d.percentage*100,2)).show(5)

+-------+-----+----------+
country|count|percentage|
+-------+-----+----------+
 PRT|48586| 40.86|
 GBR|12129| 10.2|
 FRA|10415| 8.76|
 ESP| 8568| 7.21|
 DEU| 7287| 6.13|
+-------+-----+----------+
only showing top 5 rows

Około 41% klientów hoteli pochodzi z Portugalii, 10% z Wielkiej Brytanii, a niecałe 9% z Francji.

#### Wpływ posiadania dzieci na rezerwację

In [0]:
display(data.groupby('children').count().sort('count', ascending=False))

children,count
0,110319
1,4852
2,3650
3,76
10,1


Wśród gości hoteli znacząco dominują osoby bez dzieci.

In [0]:
data = data.withColumn('has_children', f.when(data.children != 0, 'No').otherwise('Yes'))
data.groupby('has_children','canceled').count().sort('has_children').show()

+------------+--------+-----+
has_children|canceled|count|
+------------+--------+-----+
 No| No| 5453|
 No| Yes| 3126|
 Yes| Yes|41027|
 Yes| No|69292|
+------------+--------+-----+

Powyższe dane nie wskazują na istnienie jakiegokolwiek związku pomiędzy liczbą dzieci, a odwołaniem rezerwacji

In [0]:
data.createOrReplaceTempView('data')

#### Analiza klientów, którzy już wcześniej dokonali rezygnacji z rezerwacji

In [0]:
#z uwagi na dużą ilość różnej ilości poprzednich rezygnacji dodaje zmienną binarną mowiącą o tym czy ktoś kiedyś zrezygnował czy nie
data = data.withColumn('has_cancellations', f.when(data.previous_cancellations != 0, 'Yes').otherwise('No'))
data.groupby('has_cancellations','Canceled').count().sort('has_cancellations').show()

+-----------------+--------+-----+
has_cancellations|Canceled|count|
+-----------------+--------+-----+
 No| No|74216|
 No| Yes|38235|
 Yes| Yes| 5918|
 Yes| No| 529|
+-----------------+--------+-----+

Można zaobserwować wyraźną zależność pomiędzy faktem czy dana osoba wcześniej zrezygnowała z pobytu, a rezygnacją obecnie.

In [0]:
display(data.filter(data.has_cancellations == 'Yes').groupby('canceled').count())
display(data.filter(data.has_cancellations == 'No').groupby('canceled').count())

canceled,count
No,529
Yes,5918


canceled,count
No,74216
Yes,38235


92% transakcji rejestrowanych przez osoby kiedyś rezygnujące, kończy się rezygnacją z pobytu. W przypadku pozostałych klientów odsetek ten wynosi 34%.

#### Wpływ miesiąca pobytu na odwołanie rezerwacji

In [0]:
data.groupby('arrival_date_month','canceled').count().sort('arrival_date_month').show(5)

+------------------+--------+-----+
arrival_date_month|canceled|count|
+------------------+--------+-----+
 April| No| 6533|
 April| Yes| 4512|
 August| Yes| 5234|
 August| No| 8618|
 December| Yes| 2363|
+------------------+--------+-----+
only showing top 5 rows

In [0]:
%sql
create table months_data as select arrival_date_month as month, count(*) as all_reservations, sum(case when canceled == "Yes" then 1 else 0 end) as cancelled_reservations
from data
group by month

In [0]:
data_months = spark.sql('select * from months_data')
data_months.show()

+---------+----------------+----------------------+
 month|all_reservations|cancelled_reservations|
+---------+----------------+----------------------+
 July| 12628| 4736|
 November| 6752| 2120|
 February| 8012| 2695|
 January| 5874| 1806|
 March| 9739| 3148|
 October| 11095| 4228|
 May| 11779| 4677|
 August| 13852| 5234|
 April| 11045| 4512|
 June| 10927| 4534|
 December| 6728| 2363|
September| 10467| 4100|
+---------+----------------+----------------------+

In [0]:
data_months = data_months.withColumn('cancel_percentage', f.round(data_months.cancelled_reservations/data_months.all_reservations*100,2))
display(data_months.sort('cancel_percentage', ascending=False))

month,all_reservations,cancelled_reservations,cancel_percentage
June,10927,4534,41.49
April,11045,4512,40.85
May,11779,4677,39.71
September,10467,4100,39.17
October,11095,4228,38.11
August,13852,5234,37.79
July,12628,4736,37.5
December,6728,2363,35.12
February,8012,2695,33.64
March,9739,3148,32.32


Największy odsetek odwołanych rezerwacji - około 41,5% jest w czerwcu, najmniejszy natomiast (niecałe 31%) w styczniu.

#### Rezygnacje wśród powtórnych gości hotelu

In [0]:
display(data.filter(data.is_repeated_guest == 1).groupby('canceled').count())

canceled,count
No,3254
Yes,552


Innym istotnym czynnikiem wydaje się być też fakt, czy dana osoba była juz gościem danego hotelu. Z powyższego wykresu wynika, że wśród takich klientów jedynie 15% odwołuje swoją rezerwację.

In [0]:
display(data.head(5))

hotel,canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,has_children,has_cancellations
Resort Hotel,No,342,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,0,Transient,0.0,0,0,Check-Out,2015-07-01,Yes,No
Resort Hotel,No,737,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,0,Transient,0.0,0,0,Check-Out,2015-07-01,Yes,No
Resort Hotel,No,7,2015,July,27,1,0,1,1,0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,0,Transient,75.0,0,0,Check-Out,2015-07-02,Yes,No
Resort Hotel,No,13,2015,July,27,1,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,0,Transient,75.0,0,0,Check-Out,2015-07-02,Yes,No
Resort Hotel,No,14,2015,July,27,1,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,0,Transient,98.0,0,1,Check-Out,2015-07-03,Yes,No


#### Rodzaj depozytu, a ilość rezygnacji

In [0]:
%sql
create table deposit_type as select deposit_type, count(*) all_reservations, sum(case when canceled == "Yes" then 1 else 0 end) as canceled_num
from data
group by deposit_type

In [0]:
deposit_data = spark.sql('select * from deposit_type')
deposit_data = deposit_data.withColumn('cancel_percentage', f.round(deposit_data.canceled_num/deposit_data.all_reservations*100,2))
display(deposit_data.sort('cancel_percentage', ascending=False))

deposit_type,all_reservations,canceled_num,cancel_percentage
Non Refund,14573,14480,99.36
No Deposit,104163,29637,28.45
Refundable,162,36,22.22


Fakt, że prawie wszystkie rezerwacje o charakterze Non Refund są odwołane jest dosyć nietypowy. Dotarłem do dyskusji na ten temat w ramach kaggle (https://www.kaggle.com/jessemostipak/hotel-booking-demand/discussion/131787). Prawdopodobną przyczyną takiego stanu rzeczy jest wpisywanie przez potencjalnych gości nieprawidłowych danych kart kredytowych przy dokonywaniu rezerwacji. Potrzebują oni rezerwacji jedynie w celu otrzymania wizy, aby mogli dostać się do Portugalii. W momencie w którym hotel próbuje ściągnąć pieniądze, okazuje się że jest to niemożliwe i rezerwacja zostaje odwołana.

####Czas pomiędzy rezerwacją, a rozpoczęciem pobytu

In [0]:
data.select('lead_time').describe().show()

+-------+------------------+
summary| lead_time|
+-------+------------------+
 count| 118898|
 mean|104.31143501152248|
 stddev|106.90330923721025|
 min| 0|
 max| 737|
+-------+------------------+

In [0]:
#kategoryzacja zmiennej 
def categorize_lead_time(lead_time):
  if lead_time <= 10:
    return '<= 10'
  elif lead_time <= 60:
    return '11-60'
  elif lead_time <= 160:
    return '60-160'
  else:
    return '> 160'

udf_categorize_lead_time = udf(categorize_lead_time, StringType())

In [0]:
data = data.withColumn('lead_time_level', udf_categorize_lead_time(data.lead_time))
data.select('lead_time', 'lead_time_level').show(5)

+---------+---------------+
lead_time|lead_time_level|
+---------+---------------+
 342| > 160|
 737| > 160|
 7| <= 10|
 13| 11-60|
 14| 11-60|
+---------+---------------+
only showing top 5 rows

In [0]:
data.groupby('lead_time_level').count().show()

+---------------+-----+
lead_time_level|count|
+---------------+-----+
 <= 10|22506|
 60-160|33852|
 > 160|29779|
 11-60|32761|
+---------------+-----+

In [0]:
data.createOrReplaceTempView('data')

In [0]:
%sql
create table lead_time_data as select lead_time_level, count(*) all_reservations, sum(case when canceled == "Yes" then 1 else 0 end) as canceled_num
from data
group by lead_time_level

In [0]:
lead_time_data = spark.sql('select * from lead_time_data')
lead_time_data = lead_time_data.withColumn('cancel_percentage', f.round(lead_time_data.canceled_num/lead_time_data.all_reservations*100,2))
display(lead_time_data.sort('cancel_percentage', ascending=False))

lead_time_level,all_reservations,canceled_num,cancel_percentage
> 160,29779,16506,55.43
60-160,33852,14324,42.31
11-60,32761,10784,32.92
<= 10,22506,2539,11.28


Na powyższym wykresie można zaobserwować wyraźny trend - wraz ze wzrostem liczby dni pomiędzy dniem rezerwacji, a dniem rozpoczęcia pobytu, rośnie też odsetek osób odwołujących rezerwację.

###Podsumowanie

Powyższa analiza potwierdza, że istnieją czynniki, które zwiększają prawdopodobieństwo, że dana rezerwacja zostanie odwołana. Wśród przeanalizowanych znalazł się między innymi miesiąc pobytu, dla którego okazało się że okresie wiosennym od kwietnia do czerwca ryzyko odwołania rezerwacji jest najwyższe, natomiast najniższe w styczniu. Kolejnym istotnym wskaźnikiem okazał się fakt, czy rezerwujący już wcześniej odwołał rezerwację oraz czy był już gościem danego hotelu. Ci pierwsi mają znacznie większą szansę na odwołanie rezerwacji ponownie, a drudzy natomiast mniejszą. Interesujący wyniki dała również analiza zmiennej *lead_time*, która wykazała, że wraz ze wzrostem czasu pomiędzy rezerwacją, a datą przyjazdu rośnie również szansa na odwołanie rezerwacji.